# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [49]:
%pip install duckdb leafmap lonboard
import duckdb
import leafmap

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [83]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True, overwrite=True)

Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 25.4MB/s]


Extracting files...


'/content/nyc_data.db.zip'

In [84]:
con = duckdb.connect("nyc_data.db")

In [85]:
con.install_extension("spatial")
con.load_extension("spatial")

In [86]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [96]:
con.sql("""
SELECT s.name, s.routes
FROM nyc_subway_stations s
JOIN nyc_neighborhoods n
ON ST_Within(s.geom, n.geom)
WHERE n.name = 'Little Italy';
""")

┌───────────┬─────────┐
│   NAME    │ ROUTES  │
│  varchar  │ varchar │
├───────────┼─────────┤
│ Spring St │ 6       │
└───────────┴─────────┘

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [98]:
con.sql("""
SELECT DISTINCT n.name, n.boroname
FROM nyc_subway_stations s
JOIN nyc_neighborhoods n
ON ST_Within(s.geom, n.geom)
WHERE strpos(s.routes,'6') > 0;
""")

┌────────────────────┬───────────┐
│        NAME        │ BORONAME  │
│      varchar       │  varchar  │
├────────────────────┼───────────┤
│ Financial District │ Manhattan │
│ Little Italy       │ Manhattan │
│ Upper East Side    │ Manhattan │
│ East Harlem        │ Manhattan │
│ Mott Haven         │ The Bronx │
│ Hunts Point        │ The Bronx │
│ South Bronx        │ The Bronx │
│ Chinatown          │ Manhattan │
│ Greenwich Village  │ Manhattan │
│ Gramercy           │ Manhattan │
│ Murray Hill        │ Manhattan │
│ Midtown            │ Manhattan │
│ Yorkville          │ Manhattan │
│ Soundview          │ The Bronx │
│ Parkchester        │ The Bronx │
├────────────────────┴───────────┤
│ 15 rows              2 columns │
└────────────────────────────────┘

3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [100]:
con.sql("""
SELECT Sum(popn_total)
FROM nyc_neighborhoods AS n
JOIN nyc_census_blocks AS c
ON ST_Intersects(n.geom, c.geom)
WHERE n.name = 'Battery Park';
""")

┌─────────────────┐
│ sum(popn_total) │
│     int128      │
├─────────────────┤
│           17153 │
└─────────────────┘

4. **What neighborhood has the highest population density (persons/km2)?**


In [101]:
con.sql("""
SELECT
  n.name,
  Sum(c.popn_total) / (ST_Area(n.geom) / 1000000.0) AS popn_per_sqkm
FROM nyc_census_blocks AS c
JOIN nyc_neighborhoods AS n
ON ST_Intersects(c.geom, n.geom)
GROUP BY n.name, n.geom
ORDER BY popn_per_sqkm DESC LIMIT 2;
""")

┌───────────────────┬───────────────────┐
│       NAME        │   popn_per_sqkm   │
│      varchar      │      double       │
├───────────────────┼───────────────────┤
│ North Sutton Area │ 68435.13283772678 │
│ East Village      │ 50404.48341332535 │
└───────────────────┴───────────────────┘

When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [102]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00


In [103]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [104]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [114]:
url = "https://storage.googleapis.com/qm2/casa0025_ships.csv"
leafmap.download_file(url, unzip=True)

casa0025_ships.csv already exists. Skip downloading. Set overwrite=True to overwrite.


'/content/casa0025_ships.csv'

In [115]:
df = pd.read_csv(url)

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.